In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!nvidia-smi

Mon Apr 29 23:17:05 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 105...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P0    19W /  N/A |    602MiB /  4040MiB |     10%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'

In [5]:
import tensorflow as tf
config = tf.ConfigProto(intra_op_parallelism_threads=0, 
                        inter_op_parallelism_threads=0, 
                        allow_soft_placement=True)
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [6]:
import random
import numpy as np

seed = 11

np.random.seed(seed)
random.seed(seed)
tf.random.set_random_seed(seed)

In [7]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_train import simple_one_layer_cross_entropy_train
from fewshot.algorithms.fewshot_train import baseline_fewshot_train
from fewshot.algorithms.fewshot_models import BaselineFewShotModel

import tqdm
from scipy import stats

## Initialize

Data parameters:

In [8]:
img_width = 84
img_height = 84
img_depth = 3

In [9]:
dataset_root = "../fewshot/datasets/"
dataset_name = 'cub'
csv_name = "data_84x84.csv"

base_num_classes = 100
novel_num_classes = 50
val_num_classes = 50  # not used in baseline algo

Backbone training parameters:

In [10]:
# backbone training
backbone_training_batch_size = 16
backbone_training_epochs = 20

backbone_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

Few-shot training parameters:

In [11]:
# fewshot training and testing
n_way = 5
k_shot = 5
fewshot_batch_size = 4
support_epochs = 100
query_size = 16

support_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

n_episodes = 10

In [12]:
checkpoint_dir = '../fewshot/checkpoints'
log_dir = '../fewshot/logs'

## Data

Create dataset:

In [13]:
dataset_dir = os.path.join(dataset_root, dataset_name)
dataset = Dataset(dataset_dir=dataset_dir, 
                  csv_name=csv_name, 
                  image_size=(img_width, img_height))

Create backbone dataset:

In [14]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 100 (seed = 11)
Train classes: 100
Test classes: 100
Train data: 3043 samples
Test data:  2990 samples


/home/anna/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Create few-shot dataset:

In [15]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 50 (seed = 11)
Train classes: 50
Test classes: 50
Train data: 1510 samples
Test data:  1480 samples


## Backbone

In [16]:
model_name = 'baseline'

Create model:

In [17]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))
backbone.set_trainable(True)

Instructions for updating:
Colocations handled automatically by placer.


Train model:

In [18]:
backbone_optimizer = tf.keras.optimizers.Adam(lr=1e-3)

In [19]:
simple_one_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                         shuffle=True,
                                         generator_args=backbone_generator_args),
    epochs=backbone_training_epochs,
    optimizer=backbone_optimizer,
    model_name=model_name,
    checkpoint_dir=checkpoint_dir,
    period=10,
    tensorboard=True,
    log_dir=log_dir,
    use_multiprocessing=True,
    workers=4,
    verbose=2
)

Epoch 1/20
 - 6s - loss: 5.1740 - acc: 0.0187
Epoch 2/20
 - 5s - loss: 4.5307 - acc: 0.0322
Epoch 3/20
 - 5s - loss: 4.2790 - acc: 0.0463
Epoch 4/20
 - 4s - loss: 4.0355 - acc: 0.0841
Epoch 5/20
 - 5s - loss: 3.8210 - acc: 0.1163
Epoch 6/20
 - 5s - loss: 3.6302 - acc: 0.1492
Epoch 7/20
 - 5s - loss: 3.3649 - acc: 0.1936
Epoch 8/20
 - 5s - loss: 3.1882 - acc: 0.2264
Epoch 9/20
 - 5s - loss: 2.9931 - acc: 0.2682
Epoch 10/20
 - 5s - loss: 2.7516 - acc: 0.3112
Epoch 11/20
 - 4s - loss: 2.5269 - acc: 0.3575
Epoch 12/20
 - 5s - loss: 2.3384 - acc: 0.4081
Epoch 13/20
 - 5s - loss: 2.1077 - acc: 0.4482
Epoch 14/20
 - 5s - loss: 1.9068 - acc: 0.5058
Epoch 15/20
 - 5s - loss: 1.6749 - acc: 0.5633
Epoch 16/20
 - 5s - loss: 1.5246 - acc: 0.5991
Epoch 17/20
 - 5s - loss: 1.3660 - acc: 0.6441
Epoch 18/20
 - 7s - loss: 1.2215 - acc: 0.6806
Epoch 19/20
 - 4s - loss: 1.0290 - acc: 0.7351
Epoch 20/20
 - 5s - loss: 0.9024 - acc: 0.7631


## Few-shot

In [20]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=k_shot, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args)

Train few-shot model:

In [21]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 

In [22]:
fewshot_model = BaselineFewShotModel(backbone, n_way)
accuracies = baseline_fewshot_train(model=fewshot_model,
                                    generator=episode_generator, 
                                    optimizer=fewshot_optimizer,
                                    batch_size=fewshot_batch_size,
                                    support_epochs=support_epochs,
                                    n_episodes=n_episodes,
                                    model_name='baseline-fewshot',
                                    tensorboard=True,
                                    log_dir=log_dir,
                                    period=10
                                   )

  0%|          | 0/10 [00:00<?, ?it/s]

Instructions for updating:
Use tf.cast instead.


Average acc: 49.00%: 100%|██████████| 10/10 [00:31<00:00,  3.24s/it]


## Report

Calculate 95% confidence interval:

In [23]:
acc_int = stats.t.interval(0.95, len(accuracies) - 1, loc=np.mean(accuracies), scale=stats.sem(accuracies))
print("5-shot test metric: {}% +- {}%".format(round(np.mean(accuracies) * 100, 2),
                                              round((np.mean(acc_int) - acc_int[0]) * 100, 2)))

5-shot test metric: 49.0% +- 6.4%
